In [0]:
from pyspark.sql.functions import col, to_timestamp, when, concat_ws

In [0]:
#ler bronze
df_partidas_bronze = spark.table("project_data_football_bronze.partidas_rodada")

#dimensões
dim_rodada = spark.table("project_data_football_silver.dim_rodada")
dim_estadio = spark.table("project_data_football_silver.dim_estadio")
dim_campeonato = spark.table("project_data_football_silver.dim_campeonato")


In [0]:
%sql
SELECT *
FROM project_data_football_bronze.partidas_rodada

In [0]:
%sql
DESCRIBE project_data_football_bronze.partidas_rodada;

In [0]:
# garantir que contenha o ID da rodada referenciando a tabela dim_rodada na tabela fato
df = df_partidas_bronze.join(
    dim_rodada,
    df_partidas_bronze["rodada"] == dim_rodada["numero_rodada"],
    "left"
)
# garantir que contenha o ID do estadiio referenciando a tabela dim_estadio na tabela fato
df = df.join(
    dim_estadio,
    df["local"] == dim_estadio["nome_estadio"],
    "left"
)

In [0]:
# transformar para silver
df_partidas_silver = df.select(
    
    # Identificação
    col("partida_id").cast("long"),
    col("campeonato_id").cast("long"),
    col("rodada_id").cast("int"),
    col("estadio_id").cast("int"),

    # Clubes
    col("clube_casa_id").cast("long"),
    col("clube_visitante_id").cast("long"),
    
    # Placar
    col("placar_oficial_mandante").alias("gols_casa").cast("int"),
    col("placar_oficial_visitante").alias("gols_visitante").cast("int"),
    
    # Aproveitamento
    concat_ws(",", col("aproveitamento_mandante")).alias("aproveitamento_mandante"),
    concat_ws(",", col("aproveitamento_visitante")).alias("aproveitamento_visitante"),
    
    # Datas
    to_timestamp("partida_data").alias("data_partida"),
    
    # Flags
    col("valida").cast("boolean"),
)


In [0]:
# colunas análiticas extras
df_partidas_silver = df_partidas_silver.withColumn(
    "resultado",
    when(col("gols_casa") > col("gols_visitante"), "MANDANTE")
    .when(col("gols_casa") < col("gols_visitante"), "VISITANTE")
    .otherwise("EMPATE")
)

In [0]:
df_partidas_silver.filter(
    (col("rodada_id").isNull()) |
    (col("estadio_id").isNull())
).display()

In [0]:
#salvar
df_partidas_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("project_data_football_silver.fato_partida")


In [0]:
%sql
SELECT *
FROM project_data_football_silver.fato_partida